# Transfer Markt Scraping - Summer 19/20 Transfers

## Import libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

## Scraping function definition

- Input: numbers of pages to scrap (default = 1)
- Output: list of transfers

In [2]:
def Scraping_TransferMarkt_SummerTransfers(pages = 1):
    
    data = []
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    url = 'https://www.transfermarkt.com/transfers/sommertransfers/statistik?ajax=yw1&altersklasse=&ausrichtung=&land_id=0&leihe=&plus=1&sort=abloese.desc&spielerposition_id='

    for page in list(range(1, pages + 1)):

        page = url + '&page=' + str(page)
        tree = requests.get(page, headers = headers)
        soup = BeautifulSoup(tree.content, 'html.parser')
        table = soup.find('table', attrs = {'class':'items'})
        table_body = table.find('tbody')
        rows = table_body.findAll('tr')

        for row in rows[::7]:
            
            cols = row.findAll('td')
            cols = [ele.text.strip() for ele in cols]
            links = row.find_all('a', href = True)
            links = [ele['href'].split('/')[-1] for ele in links]
            countries = row.find_all('img', alt = True)[-4:]
            countries = [ele['alt'] for ele in countries]
            joinedlist = cols + links + countries
            data.append([ele for ele in joinedlist if ele]) # Get rid of empty values
        
        time.sleep(.5)
    
    return data

## Execute scraping function and export raw data to CSV file

In [3]:
#df = pd.DataFrame(Scraping_TransferMarkt_SummerTransfers(324))
#df.to_csv('./Output Files/Preprocessed_Transfers.csv', index = False)

## Import scraped data

In [4]:
df = pd.read_csv('./Intermediate Files/Preprocessed_Transfers.csv')
df.head(n = 2)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,João Félix \n\n\nSecond Striker,João Félix,Second Striker,19,"70,00 mil. €",Benfica \n\n\n\n Liga NOS,Benfica,Liga NOS,Atlético Madrid \n\n\n\n LaLiga,...,294,294,PO1,13,13,ES1,SL Benfica,Portugal,Atlético Madrid,Spain
1,2,Antoine Griezmann \n\n\nCentre-Forward,Antoine Griezmann,Centre-Forward,28,"130,00 mil. €",Atlético Madrid \n\n\n\n LaLiga,Atlético Madrid,LaLiga,FC Barcelona \n\n\n\n LaLiga,...,13,13,ES1,131,131,ES1,Atlético Madrid,Spain,FC Barcelona,Spain


In [5]:
df.drop(df.columns[[1,6,9,15,18]], axis = 1, inplace = True)
df.head(n = 2)

,0,2,3,4,5,7,8,10,11,12,13,14,16,17,19,20,21,22,23
0,1,João Félix,Second Striker,19,"70,00 mil. €",Benfica,Liga NOS,Atlético Madrid,LaLiga,"126,00 mil. €",462250,294,PO1,13,ES1,SL Benfica,Portugal,Atlético Madrid,Spain
1,2,Antoine Griezmann,Centre-Forward,28,"130,00 mil. €",Atlético Madrid,LaLiga,FC Barcelona,LaLiga,"120,00 mil. €",125781,13,ES1,131,ES1,Atlético Madrid,Spain,FC Barcelona,Spain


## Define column names

In [6]:
df.columns = ['ID', 'Player', 'Position', 'Age', 'MarketValue',
              'TeamLeft', 'LeagueLeft', 'TeamJoined', 'LeagueJoined', 'TransferFee', 
              'PlayerID', 'TeamLeftID', 'LeagueLeftID', 'TeamJoinedID', 'LeagueJoinedID',
              'TeamLeft2', 'CountryLeft', 'TeamJoined2', 'CountryJoined']
df.head(n = 2)

,ID,Player,Position,Age,MarketValue,TeamLeft,LeagueLeft,TeamJoined,LeagueJoined,TransferFee,PlayerID,TeamLeftID,LeagueLeftID,TeamJoinedID,LeagueJoinedID,TeamLeft2,CountryLeft,TeamJoined2,CountryJoined
0,1,João Félix,Second Striker,19,"70,00 mil. €",Benfica,Liga NOS,Atlético Madrid,LaLiga,"126,00 mil. €",462250,294,PO1,13,ES1,SL Benfica,Portugal,Atlético Madrid,Spain
1,2,Antoine Griezmann,Centre-Forward,28,"130,00 mil. €",Atlético Madrid,LaLiga,FC Barcelona,LaLiga,"120,00 mil. €",125781,13,ES1,131,ES1,Atlético Madrid,Spain,FC Barcelona,Spain


In [7]:
df.set_index('ID', inplace = True)
df.head(n = 2)

,Player,Position,Age,MarketValue,TeamLeft,LeagueLeft,TeamJoined,LeagueJoined,TransferFee,PlayerID,TeamLeftID,LeagueLeftID,TeamJoinedID,LeagueJoinedID,TeamLeft2,CountryLeft,TeamJoined2,CountryJoined
ID,,,,,,,,,,,,,,,,,,
1,João Félix,Second Striker,19,"70,00 mil. €",Benfica,Liga NOS,Atlético Madrid,LaLiga,"126,00 mil. €",462250,294,PO1,13,ES1,SL Benfica,Portugal,Atlético Madrid,Spain
2,Antoine Griezmann,Centre-Forward,28,"130,00 mil. €",Atlético Madrid,LaLiga,FC Barcelona,LaLiga,"120,00 mil. €",125781,13,ES1,131,ES1,Atlético Madrid,Spain,FC Barcelona,Spain


## Data cleaning

- Remove loans

In [8]:
df = df.loc[~df['TransferFee'].str.startswith('Loan', na = False)]

- Remove invalid and unknown transfer fees

In [9]:
df = df.loc[~df['TransferFee'].str.startswith('draft', na = False)]
df = df.loc[~df['TransferFee'].str.startswith('?', na = False)]
df = df.loc[~df['TransferFee'].str.startswith('-', na = False)]

### Function definition: Conversion from string monetary to floating number

In [10]:
def currency_string_to_number(value):
   
    multiplier = 1
    number_delimiter_position = value.find(' ')
    y = value.replace(',','.')
     
    if value == 'Free transfer':
        return 0.0
    elif value[-6:] == 'mil. €':
        multiplier = 1000000
        return float(y[:number_delimiter_position])*multiplier
    elif value[-10:] == 'thousand €':
        multiplier = 1000
        return float(y[:number_delimiter_position])*multiplier
    elif value[-3:] == 'K €':
        multiplier = 1000
        return float(y[:number_delimiter_position])*multiplier
    elif value == '-':
        return 0.0
    else:
        return(float(value))

- Apply conversion function to Transfer Fees

In [11]:
df['TransferFee'] = df['TransferFee'].apply(currency_string_to_number)

- Remove invalid market value entries

In [12]:
df = df[df['MarketValue'] != 'Loros \n\n\n\n Ascenso MX Apertura']

- Apply conversion function to Market Value

In [13]:
df['MarketValue'] = df['MarketValue'].apply(currency_string_to_number)

- Filter only on Major Leagues

In [14]:
FIFA_leagues = ['A1','AR1N','AUS1','BE1','BRA1','C1','C2','DK1',
                'ES1','ES2','FI1','FR1','FR2','GB1','GB2','GB3',
                'GB4','GR1','IT1','IT2','JAP1','KR1','L1','L2','L3',
                'MEXA','MLS1','NL1','NO1','PL1','PO1','RO1','RU1',
                'SC1','SE1','SFA1','TR1','TS1','UAE1']

In [15]:
df = df.loc[ (df['LeagueLeftID'].isin(FIFA_leagues)) & (df['LeagueJoinedID'].isin(FIFA_leagues)), : ]

- Clean player positions

In [16]:
df.Position.value_counts()

Centre-Back           327
Centre-Forward        297
Central Midfield      205
Right-Back            164
Left-Back             137
Defensive Midfield    128
Goalkeeper            128
Left Winger           120
Right Winger          118
Attacking Midfield    104
Right Midfield         32
Left Midfield          30
Second Striker         21
Defender                1
Name: Position, dtype: int64

In [17]:
df = df[df['Position'] != 'Defender']

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1811 entries, 1 to 6999
Data columns (total 18 columns):
Player            1811 non-null object
Position          1811 non-null object
Age               1811 non-null object
MarketValue       1811 non-null float64
TeamLeft          1811 non-null object
LeagueLeft        1811 non-null object
TeamJoined        1811 non-null object
LeagueJoined      1811 non-null object
TransferFee       1811 non-null float64
PlayerID          1811 non-null int64
TeamLeftID        1811 non-null object
LeagueLeftID      1811 non-null object
TeamJoinedID      1811 non-null object
LeagueJoinedID    1811 non-null object
TeamLeft2         1811 non-null object
CountryLeft       1811 non-null object
TeamJoined2       1811 non-null object
CountryJoined     1811 non-null object
dtypes: float64(2), int64(1), object(15)
memory usage: 268.8+ KB


- Change column data types

In [19]:
df['Age'] = df['Age'].astype(int)
df['MarketValue'] = df['MarketValue'].astype(int)
df['TransferFee'] = df['TransferFee'].astype(int)
df['PlayerID'] = df['PlayerID'].astype(int)
df['TeamLeftID'] = df['TeamLeftID'].astype(int)
df['TeamJoinedID'] = df['TeamJoinedID'].astype(int)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1811 entries, 1 to 6999
Data columns (total 18 columns):
Player            1811 non-null object
Position          1811 non-null object
Age               1811 non-null int64
MarketValue       1811 non-null int64
TeamLeft          1811 non-null object
LeagueLeft        1811 non-null object
TeamJoined        1811 non-null object
LeagueJoined      1811 non-null object
TransferFee       1811 non-null int64
PlayerID          1811 non-null int64
TeamLeftID        1811 non-null int64
LeagueLeftID      1811 non-null object
TeamJoinedID      1811 non-null int64
LeagueJoinedID    1811 non-null object
TeamLeft2         1811 non-null object
CountryLeft       1811 non-null object
TeamJoined2       1811 non-null object
CountryJoined     1811 non-null object
dtypes: int64(6), object(12)
memory usage: 268.8+ KB


- Reorder columns

In [21]:
df = df[['PlayerID', 'Player', 'Position', 'Age', 'MarketValue', 'TransferFee',
        'TeamLeftID', 'TeamLeft', 'TeamLeft2', 'LeagueLeftID', 'LeagueLeft', 'CountryLeft',
        'TeamJoinedID', 'TeamJoined', 'TeamJoined2', 'LeagueJoinedID', 'LeagueJoined', 'CountryJoined']]

In [22]:
df.head(n = 10)

,PlayerID,Player,Position,Age,MarketValue,TransferFee,TeamLeftID,TeamLeft,TeamLeft2,LeagueLeftID,LeagueLeft,CountryLeft,TeamJoinedID,TeamJoined,TeamJoined2,LeagueJoinedID,LeagueJoined,CountryJoined
ID,,,,,,,,,,,,,,,,,,
1,462250,João Félix,Second Striker,19,70000000,126000000,294,Benfica,SL Benfica,PO1,Liga NOS,Portugal,13,Atlético Madrid,Atlético Madrid,ES1,LaLiga,Spain
2,125781,Antoine Griezmann,Centre-Forward,28,130000000,120000000,13,Atlético Madrid,Atlético Madrid,ES1,LaLiga,Spain,131,FC Barcelona,FC Barcelona,ES1,LaLiga,Spain
3,50202,Eden Hazard,Left Winger,28,150000000,100000000,631,Chelsea,Chelsea FC,GB1,Premier League,England,418,Real Madrid,Real Madrid,ES1,LaLiga,Spain
4,177907,Harry Maguire,Centre-Back,26,50000000,87000000,1003,Leicester,Leicester City,GB1,Premier League,England,985,Man Utd,Manchester United,GB1,Premier League,England
5,326031,Matthijs de Ligt,Centre-Back,20,75000000,85500000,610,Ajax,Ajax Amsterdam,NL1,Eredivisie,Netherlands,506,Juventus,Juventus FC,IT1,Serie A,Italy
6,281963,Lucas Hernández,Centre-Back,23,70000000,80000000,13,Atlético Madrid,Atlético Madrid,ES1,LaLiga,Spain,27,Bayern Munich,Bayern Munich,L1,Bundesliga,Germany
7,343052,Nicolas Pépé,Right Winger,24,65000000,80000000,1082,LOSC Lille,LOSC Lille,FR1,Ligue 1,France,11,Arsenal,Arsenal FC,GB1,Premier League,England
8,326330,Frenkie de Jong,Central Midfield,22,85000000,75000000,610,Ajax,Ajax Amsterdam,NL1,Eredivisie,Netherlands,131,FC Barcelona,FC Barcelona,ES1,LaLiga,Spain
9,357565,Rodri,Defensive Midfield,23,80000000,70000000,13,Atlético Madrid,Atlético Madrid,ES1,LaLiga,Spain,281,Man City,Manchester City,GB1,Premier League,England


## Export results to CSV file

In [23]:
df.to_csv('./Output Files/Transfers.csv', index = True)